In [1]:
import math
import random
import librosa
import numpy as np
import parselmouth
import scipy.signal
import torch
import torchaudio.functional as AF

import function_f

import os
import pickle
import soundfile as sf
from scipy import signal
from scipy.signal import get_window
from librosa.filters import mel

In [2]:
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a
    
    
def pySTFT(x, fft_length=1024, hop_length=256):
    
    x = np.pad(x, int(fft_length//2), mode='reflect')
    
    noverlap = fft_length - hop_length
    shape = x.shape[:-1]+((x.shape[-1]-noverlap)//hop_length, fft_length)
    strides = x.strides[:-1]+(hop_length*x.strides[-1], x.strides[-1])
    result = np.lib.stride_tricks.as_strided(x, shape=shape,
                                             strides=strides)
    
    fft_window = get_window('hann', fft_length, fftbins=True)
    result = np.fft.rfft(fft_window * result, n=fft_length).T
    
    return np.abs(result)    
    
    
mel_basis = mel(sr=16000, n_fft=1024, fmin=90, fmax=7600, n_mels=80).T
min_level = np.exp(-100 / 20 * np.log(10))
b, a = butter_highpass(30, 16000, order=5)

In [3]:
rootDir = '/data/hypark/VC/data/VCTK/wav48'
targetDir = './preprocess_data/VCTK/perturb_f'

In [4]:
import soundfile as sf
from tqdm.auto import tqdm


dirName, subdirList, _ = next(os.walk(rootDir))
print('Found directory: %s' % dirName)

for subdir in sorted(subdirList):
    print(subdir)
    if subdir == '.ipynb_checkpoints':
        continue
    if not os.path.exists(os.path.join(targetDir, subdir)):
        os.makedirs(os.path.join(targetDir, subdir))
    _, _, fileList = next(os.walk(os.path.join(dirName,subdir)))
    #prng = RandomState(int(subdir[1:])) 
    for fileName in tqdm(sorted(fileList), total=len(fileList)):
        x, fs = librosa.load(os.path.join(dirName,subdir,fileName), sr=16000)
        x = function_f.f(torch.from_numpy(x), sr=fs)
        if not os.path.exists(os.path.join(targetDir, subdir)):
            os.makedirs(os.path.join(targetDir, subdir))
        # sf.write(os.path.join(targetDir, subdir, fileName), x, 16000)
        y = signal.filtfilt(b, a, x)
        D = pySTFT(y).T
        D_mel = np.dot(D, mel_basis)
        D_db = 20 * np.log10(np.maximum(min_level, D_mel)) - 16
        S = np.clip((D_db + 100) / 100, 0, 1)      
        np.save(os.path.join(targetDir, subdir, fileName[:-4]),
                S.astype(np.float32).T, allow_pickle=False)

Found directory: /data/hypark/VC/data/VCTK/wav48
p225


  0%|          | 0/231 [00:00<?, ?it/s]

p226


  0%|          | 0/356 [00:00<?, ?it/s]

p227


...

p345


  0%|          | 0/397 [00:00<?, ?it/s]

p347


  0%|          | 0/423 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

